# Running gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2024-11-01

In [1]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')
import xarray as xr
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

2025-01-13 15:25:09,761 - distributed.scheduler - WARNING - Detected different `run_spec` for key ('original-open_dataset-size_bins-72c7f0fba951651fb99d2c971bf417b8', 0) between two consecutive calls to `update_graph`. This can cause failures and deadlocks down the line. Please ensure unique key names. If you are using a standard dask collections, consider releasing all the data before resubmitting another computation. More details and help can be found at https://github.com/dask/dask/issues/9888. 
Debugging information
---------------------
old task state: released
old run_spec: <Task ('original-open_dataset-size_bins-72c7f0fba951651fb99d2c971bf417b8', 0) _execute_subgraph(...)>
new run_spec: <Task ('original-open_dataset-size_bins-72c7f0fba951651fb99d2c971bf417b8', 0) _execute_subgraph(...)>
old token: '362a2827097e72747b479dab2e477614'
new token: '2a06d6e4001fc62ec8acd727236b2a4c'
old dependencies: set()
new dependencies: set()

2025-01-13 15:25:40,561 - distributed.scheduler - WARN

## Loading Python friendly parameters 

In [3]:
base_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/gridded_params/'

In [4]:
sinking_rate = xr.open_zarr(
    os.path.join(
        '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/gridded', 
        'gfdl-mom6-cobalt2_obsclim_er_15arcmin_fao-58_monthly_1961_2010.zarr/'))['export_ratio']

effort = xr.open_zarr(
    glob(os.path.join(base_folder, 
                      'effort_*'))[0])['effort']
   
predators = xr.open_zarr(glob(
    os.path.join(base_folder, 
                 'predators_*'))[0])['predators'] 

detritivores = xr.open_zarr(glob(
    os.path.join(base_folder, 
                 'detritivores_*'))[0])['detritivores']

detritus = xr.open_zarr(glob(
    os.path.join(base_folder, 
                 'detritus_*'))[0])['detritus']

pel_tempeffect = xr.open_zarr(glob(
    os.path.join(base_folder, 'pel-temp-eff_*'))[0])['pel_temp_eff']

ben_tempeffect = xr.open_zarr(glob(
    os.path.join(base_folder, 'ben-temp-eff_*'))[0])['ben_temp_eff']

In [5]:
depth = xr.open_zarr(os.path.join(
    base_folder, 
    'gfdl-mom6-cobalt2_obsclim_deptho_15arcmin_fao-58_fixed.zarr/'))['deptho']
log10_size_bins_mat = xr.open_zarr('outputs/log10_size_bins_matrix.zarr/')['size_bins']

In [7]:
out_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/test_run'
os.makedirs(out_folder, exist_ok= True)

In [8]:
uf.gridded_sizemodel(base_folder, sinking_rate, depth, 
                     log10_size_bins_mat, region = 'fao-58',
                     out_folder = out_folder)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/minico

ValueError: destination buffer too small; expected at least 203520, got 7360

In [10]:
# from glob import glob
a = xr.open_dataarray(glob(os.path.join(out_folder, '*detritus*'))[0])
a
# a = xr.open_mfdataset(sorted(glob(os.path.join(out_folder, 'effo*')))

<xarray.DataArray (lat: 46, lon: 40)> Size: 15kB
[1840 values with dtype=float64]
Coordinates:
  * lat      (lat) float64 368B -60.12 -60.38 -60.62 ... -70.88 -71.12 -71.38
  * lon      (lon) float64 320B 70.12 70.38 70.62 70.88 ... 79.38 79.62 79.88
    time     datetime64[ns] 8B ...

In [11]:
a.time

<xarray.DataArray 'time' ()> Size: 8B
[1 values with dtype=datetime64[ns]]
Coordinates:
    time     datetime64[ns] 8B ...

In [34]:
b = xr.open_mfdataset(sorted(glob(os.path.join(out_folder,
                                               '*predators*'))),
                            combine='nested', concat_dim = 'time')

In [17]:
a= xr.open_zarr('/g/data/vf71/la6889/dbpm_inputs/east_antarctica/gridded_params/ben-temp-eff_15arcmin_fao-58_monthly_1961_2010.zarr/')
a = a ['ben_temp_eff']

In [36]:
# a.indexes['time'].to_datetimeindex()[0]
b.time

<xarray.DataArray 'time' (time: 3)> Size: 24B
array(['1961-01-01T00:00:00.000000000', '1961-02-01T00:00:00.000000000',
       '1961-03-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 24B 1961-01-01 1961-02-01 1961-03-01

In [28]:
b = xr.zeros_like(a[0])
b['time'] = str(a.indexes['time'].to_datetimeindex()[0])
b.time

/jobfs/132222511.gadi-pbs/ipykernel_1560886/3365293635.py:2: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  b['time'] = str(a.indexes['time'].to_datetimeindex()[0])


<xarray.DataArray 'time' ()> Size: 76B
array('1961-01-01 00:00:00', dtype='<U19')
Coordinates:
    time     <U19 76B '1961-01-01 00:00:00'

In [27]:
str(a.indexes['time'].to_datetimeindex()[0])

/jobfs/132222511.gadi-pbs/ipykernel_1560886/458749048.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  str(a.indexes['time'].to_datetimeindex()[0])


'1961-01-01 00:00:00'

In [30]:
sinking_rate.isel(time = 0).time

<xarray.DataArray 'time' ()> Size: 8B
array(cftime.Datetime360Day(1961, 1, 1, 0, 0, 0, 0, has_year_zero=True),
      dtype=object)
Coordinates:
    time     object 8B 1961-01-01 00:00:00

## Outside time loop

### Fisheries catches per year per size class
`fishing_mort_pred` and `fishing_mort_det` are set to zero outside the sizes for each class, so there is no need to apply a mask here.

In [356]:
catch_pred = fishing_mort_pred*predators*size_bin_vals
catch_det = fishing_mort_det*detritivores*size_bin_vals